In [1]:
import os
import json
import re
import pandas as pd
import numpy as np
from slugify import slugify
import frontmatter

DIRNAME = os.path.abspath('')
THETIS_CSV_PATH = os.path.join(DIRNAME, "../files_computed/thetis_all_with_computed.csv")
WIKIDATA_SHIPS_CSV_PATH = os.path.join(DIRNAME, "../files_original/original.wikidata.ships.csv")
WIKIDATA_URLS_CSV_PATH = os.path.join(DIRNAME, "../files_original/original.wikidata.urls.csv")
WWW_SHIPS_DATA_PATH = os.path.join(DIRNAME, "../../www/11ty_input/ships")

In [19]:
all_ships = []
for filename in os.listdir(WWW_SHIPS_DATA_PATH):
    if not re.match(r".*-(\d+)\.md", filename):
        continue
    path = os.path.join(WWW_SHIPS_DATA_PATH, filename)
    ship_frontmatter = frontmatter.load(path)
    all_ships.append({ "mdPath": path, **ship_frontmatter.metadata })
df = pd.DataFrame(all_ships)
df.head()

,mdPath,company,imo,layout,name,outOfScope,routes,slug,tags,photo,unknownRoutes,wikipediaUrl,capacityPax
0,/Users/adipasquale/dev/greenferries/data/noteb...,stena-line-se,9332559,ship,MS Stena Horizon,False,[cherbourg-fr-rosslare-ie],ms-stena-horizon-9332559,[ship],/img/300px-Stena_Horizon_approaching_Rosslare_...,False,https://en.wikipedia.org/wiki/MS_Stena_Horizon,NaN
1,/Users/adipasquale/dev/greenferries/data/noteb...,tui-cruises-de,9783576,ship,Mein Schiff 2,NaN,[],mein-schiff-2-9783576,[ship],/img/mein-schiff-2-9783576.jpg,NaN,NaN,NaN
2,/Users/adipasquale/dev/greenferries/data/noteb...,anek-lines-gr,7814046,ship,Q3817075,False,[],q3817075-7814046,[ship],/img/300px-20080814-Piraeus-Kriti1.jpg,True,https://fr.wikipedia.org/wiki/Kriti_I,1600.0
3,/Users/adipasquale/dev/greenferries/data/noteb...,tallink-ee,9364722,ship,MS Star,NaN,[helsinki-fi-tallinn-ee],ms-star-9364722,[ship],/img/300px-190407_Star_Helsinki.jpg,NaN,https://en.wikipedia.org/wiki/MS_Star,1900.0
4,/Users/adipasquale/dev/greenferries/data/noteb...,unity-line-pl,9108350,ship,Q11764893,False,[ystad-se-swinoujscie-pl],q11764893-9108350,[ship],/img/300px-Polonia_3_by_jakub_bogucki.jpg,False,https://pl.wikipedia.org/wiki/MF_Polonia,NaN


In [14]:
df[df.outOfScope == True]['company'].value_counts()

royal-caribbean-international-us    12
aida-cruises-de                      9
costa-cruises-it                     7
msc-cruises-it                       5
celebrity-cruises-us                 4
regent-seven-seas-cruises-us         3
princess-cruises-us                  3
hapag-lloyd-de                       3
fred-olsen-cruise-lines-no           3
azamara-club-cruises-us              2
seatrans-norway-no                   2
phoenix-reisen-de                    2
saga-cruises-gb                      2
cunard-line-gb                       2
holland-america-line-us              2
birka-line-se                        1
far-east-shipping-company-ru         1
disney-cruise-line-us                1
salamis-cruises-cy                   1
tui-cruises-de                       1
color-line-no                        1
carnival-cruise-line-us              1
trasmediterranea-es                  1
road-scholar-us                      1
cruise-maritime-voyages-gb           1
unity-line-pl            

In [27]:
cruiseCompanies = [c for c in set(df[df.outOfScope == True]['company']) if "cruise" in c]
cruiseCompanies

['saga-cruises-gb',
 'tui-cruises-de',
 'crystal-cruises-us',
 'aida-cruises-de',
 'azamara-club-cruises-us',
 'cruise-maritime-voyages-gb',
 'celebrity-cruises-us',
 'salamis-cruises-cy',
 'fred-olsen-cruise-lines-no',
 'disney-cruise-line-us',
 'kristina-cruises-fi',
 'costa-cruises-it',
 'carnival-cruise-line-us',
 'regent-seven-seas-cruises-us',
 'princess-cruises-us',
 'msc-cruises-it']

In [37]:
for _idx, row in df[df.company.isin(cruiseCompanies)][df.outOfScope != True].iterrows():
    row_dict = row.to_dict()
    ship_frontmatter = frontmatter.load(row_dict["mdPath"])
    ship_frontmatter.metadata["outOfScope"] = True
    with open(row_dict["mdPath"], "w") as f:
        f.write(frontmatter.dumps(ship_frontmatter))
